In [ ]:
pip install linearmodels

    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
from linearmodels import PooledOLS
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import numpy.linalg as la
from scipy import stats
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
def hausman(fe, re):
  b = fe.params
  B = re.params
  v_b = fe.cov
  v_B = re.cov
  df = b[np.abs(b) < 1e8].size
  chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
  pval = stats.chi2.sf(chi2, df)
  return chi2, df, pval

In [ ]:
df = pd.read_csv("Dataset.csv", index_col = ['Location','Period'])
years = df.index.get_level_values('Period').to_list()
df['Period'] = pd.Categorical(years)
df.head()

,,Suicide_per_100000,Male,Female,PSEGDP,Old_Age,Survivors,Incapacity,Family,Labour,Unemployment,Housing,Health,Other,Total_GDP,Period
Location,Period,,,,,,,,,,,,,,,
Australia,2000,12.7,19.9,5.6,18.244,5.653,0.229,2.560,2.906,0.361,0.926,0.244,5.206,0.159,8.491370e+11,2000
Austria,2000,19.9,29.8,10.6,25.688,10.083,2.228,2.704,2.929,0.504,0.922,0.101,5.924,0.292,3.364950e+11,2000
Belgium,2000,22.0,31.5,12.9,23.654,6.817,2.089,2.006,2.468,0.830,2.755,0.033,5.968,0.688,4.058330e+11,2000
Canada,2000,11.9,18.3,5.7,15.709,3.751,0.421,0.936,0.883,0.365,0.673,0.555,5.722,2.405,1.207140e+12,2000
Chile,2000,10.3,17.9,2.9,10.397,3.952,1.061,0.799,1.139,0.212,0.054,0.418,2.518,0.244,4.871490e+11,2000


In [ ]:
df_t = df.copy()
var_name = ['PSEGDP','Old_Age','Survivors','Incapacity','Family','Labour',"Unemployment",'Housing','Health','Other']
dvar_name = ['Suicide_per_100000', 'Male', 'Female']
for i in var_name:
  df_t[i] = df_t[i]*df_t['Total_GDP']/100000000
df_t.head()

,,Suicide_per_100000,Male,Female,PSEGDP,Old_Age,Survivors,Incapacity,Family,Labour,Unemployment,Housing,Health,Other,Total_GDP,Period
Location,Period,,,,,,,,,,,,,,,
Australia,2000,12.7,19.9,5.6,154916.55428,48001.71461,1944.52373,21737.90720,24675.92122,3065.38457,7863.00862,2071.89428,44206.07222,1350.12783,8.491370e+11,2000
Austria,2000,19.9,29.8,10.6,86438.83560,33928.79085,7497.10860,9098.82480,9855.93855,1695.93480,3102.48390,339.85995,19933.96380,982.56540,3.364950e+11,2000
Belgium,2000,22.0,31.5,12.9,95995.73782,27665.63561,8477.85137,8141.00998,10015.95844,3368.41390,11180.69915,133.92489,24220.11344,2792.13104,4.058330e+11,2000
Canada,2000,11.9,18.3,5.7,189629.62260,45279.82140,5082.05940,11298.83040,10659.04620,4406.06100,8124.05220,6699.62700,69072.55080,29031.71700,1.207140e+12,2000
Chile,2000,10.3,17.9,2.9,50648.88153,19252.12848,5168.65089,3892.32051,5548.62711,1032.75588,263.06046,2036.28282,12266.41182,1188.64356,4.871490e+11,2000


In [ ]:
results_all_var = pd.DataFrame(columns = ['Dependent', 'White Test P-Value', 'Breusch Pagan Test P-Value',
                                  'Durbin Watson Test P-Value','Hausman Entity vs Random', 'Hausman Time vs Random',
                                   'Hausman Entity and Time vs Random'])
for i in dvar_name:
  exog = sm.tools.tools.add_constant(df[['PSEGDP','Old_Age','Survivors','Incapacity','Family','Labour',"Unemployment",'Housing','Health','Other']])
  endog = df[i]
  mod = PooledOLS(endog, exog)
  pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)
  # Store values for checking homoskedasticity graphically
  fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
  residuals_pooled_OLS = pooledOLS_res.resids
  fig, ax = plt.subplots()
  ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS,s = 0.5, color = 'blue')
  ax.axhline(0, color = 'r', ls = '--')
  ax.set_xlabel('Predicted Values', fontsize = 15)
  ax.set_ylabel('Residuals', fontsize = 15)
  ax.set_title('Homoskedasticity Test', fontsize = 30)
  plt.close(fig)
  plt.savefig('All_Var_'+i+'.png')

  pooled_OLS_dataset = pd.concat([df, residuals_pooled_OLS], axis=1)
  pooled_OLS_dataset = pooled_OLS_dataset.drop(['Period'], axis = 1).fillna(0)
  exog = sm.tools.tools.add_constant(df[i]).fillna(0)
  white_test_results = het_white(pooled_OLS_dataset['residual'], exog)

  # 3A.3 Breusch-Pagan-Test
  breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
  durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 

  exog = sm.tools.tools.add_constant(df[['PSEGDP','Old_Age','Survivors','Incapacity','Family','Labour',"Unemployment",'Housing','Health','Other']])
  endog = df[i]
  model_re = RandomEffects(endog, exog) 
  re_res = model_re.fit() 
  print(re_res)
  # fixed effects model
  model_fe = PanelOLS(endog, exog, entity_effects = True) 
  fe_res = model_fe.fit() 
  print(fe_res)
  # time effects
  model_te = PanelOLS(endog, exog, time_effects = True)
  te_res =  model_te.fit() 
  print(te_res)
  # time effects
  model_te_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True)
  fe_te_res =  model_te_fe.fit() 
  print(fe_te_res)

  hausman_results1 = hausman(fe_res, re_res) 
  hausman_results2 = hausman(te_res, re_res) 
  hausman_results3 = hausman(fe_te_res, re_res)

  results_all_var = results_all_var.append({'Dependent' : i, 'White Test P-Value' : white_test_results[1], 'Breusch Pagan Test P-Value' : breusch_pagan_test_results[1],
                              'Durbin Watson Test P-Value' : durbin_watson_test_results,
                              'Hausman Entity vs Random' : hausman_results1[2], 'Hausman Time vs Random' : hausman_results2[2],
                              'Hausman Entity and Time vs Random' : hausman_results3[2]}, ignore_index = True)

                        RandomEffects Estimation Summary                        
Dep. Variable:     Suicide_per_100000   R-squared:                        0.0934
Estimator:              RandomEffects   R-squared (Between):             -0.0698
No. Observations:                 630   R-squared (Within):               0.1014
Date:                Tue, Jun 01 2021   R-squared (Overall):             -0.0551
Time:                        20:38:40   Log-likelihood                   -1394.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.3799
Entities:                          35   P-value                           0.0000
Avg Obs:                       18.000   Distribution:                  F(10,619)
Min Obs:                       18.000                                           
Max Obs:                       18.000   F-statistic (robust):             6.3799
                            

<Figure size 432x288 with 0 Axes>

In [ ]:
print(results_all_var)
results_all_var.to_csv('Results_all_var.csv')

            Dependent  ...  Hausman Entity and Time vs Random
0  Suicide_per_100000  ...                       2.394768e-24
1                Male  ...                       1.000000e+00
2              Female  ...                       1.152234e-65

[3 rows x 7 columns]


In [ ]:
results = pd.DataFrame(columns = ['Independent Variable', 'Dependent', 'White Test P-Value', 'Breusch Pagan Test P-Value',
                                  'Durbin Watson Test P-Value', 'Random Effect P-Values','Const R', 'I Var R', 'Fixed Effect P-Values', 'Const FE', 'I Var FE',
                                  'Time Effect P-Values','Const FT', 'I Var FT', 'Entity and Time P-Values','Const FA', 'I Var FA',
                                  'Hausman Entity vs Random', 'Hausman Time vs Random', 'Hausman Entity and Time vs Random',
                                  'Hausman Time vs Entity',
                                  'Hausman Entity and Time vs Entity','Hausman Entity and Time vs Time'])
for i in var_name:
  for j in dvar_name:
    exog = sm.tools.tools.add_constant(df[i])
    endog = df[j]
    mod = PooledOLS(endog, exog)
    pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)
    # Store values for checking homoskedasticity graphically
    fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
    residuals_pooled_OLS = pooledOLS_res.resids
    fig, ax = plt.subplots()
    ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS,s = 0.5, color = 'blue')
    ax.axhline(0, color = 'r', ls = '--')
    ax.set_xlabel('Predicted Values', fontsize = 15)
    ax.set_ylabel('Residuals', fontsize = 15)
    ax.set_title('Homoskedasticity Test', fontsize = 30)
    plt.close(fig)
    plt.savefig(i+'_'+j+'.png')

    pooled_OLS_dataset = pd.concat([df, residuals_pooled_OLS], axis=1)
    pooled_OLS_dataset = pooled_OLS_dataset.drop(['Period'], axis = 1).fillna(0)
    exog = sm.tools.tools.add_constant(df[i]).fillna(0)
    white_test_results = het_white(pooled_OLS_dataset['residual'], exog)

    # 3A.3 Breusch-Pagan-Test
    breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
    durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 

    exog = sm.tools.tools.add_constant(df[i])
    endog = df[j]
    model_re = RandomEffects(endog, exog) 
    re_res = model_re.fit() 
    # fixed effects model
    model_fe = PanelOLS(endog, exog, entity_effects = True) 
    fe_res = model_fe.fit() 
    # time effects
    model_te = PanelOLS(endog, exog, time_effects = True)
    te_res =  model_te.fit() 
    # time effects
    model_te_fe = PanelOLS(endog, exog, entity_effects = True, time_effects = True)
    fe_te_res =  model_te_fe.fit() 

    hausman_results1 = hausman(fe_res, re_res) 
    hausman_results2 = hausman(te_res, re_res) 
    hausman_results3 = hausman(fe_te_res, re_res)
    hausman_results4 = hausman(te_res, fe_res)
    hausman_results5 = hausman(fe_te_res, fe_res)
    hausman_results6 = hausman(fe_te_res, te_res)

    results = results.append({'Independent Variable' : i, 'Dependent' : j, 'White Test P-Value' : white_test_results[1], 'Breusch Pagan Test P-Value' : breusch_pagan_test_results[1],
                              'Durbin Watson Test P-Value' : durbin_watson_test_results, 'Random Effect P-Values' : re_res.pvalues[1], 'Const R' : re_res.params[0], 'I Var R' : re_res.params[1], 
                              'Fixed Effect P-Values' : fe_res.pvalues[1], 'Const FE' : fe_res.params[0], 'I Var FE' : fe_res.params[1], 'Time Effect P-Values' : te_res.pvalues[1], 
                              'Const FT' : te_res.params[0], 'I Var FT' : te_res.params[1], 'Entity and Time P-Values' : fe_te_res.pvalues[1], 'Const FA' : fe_te_res.params[0], 'I Var FA' : fe_te_res.params[1],
                              'Hausman Entity vs Random' : hausman_results1[2], 'Hausman Time vs Random' : hausman_results2[2],
                              'Hausman Entity and Time vs Random' : hausman_results3[2], 'Hausman Time vs Entity' : hausman_results4[2],
                              'Hausman Entity and Time vs Entity' : hausman_results5[2], 'Hausman Entity and Time vs Time' : hausman_results6[2]}, ignore_index = True)

<Figure size 432x288 with 0 Axes>

In [ ]:
results.head()

,Independent Variable,Dependent,White Test P-Value,Breusch Pagan Test P-Value,Durbin Watson Test P-Value,Random Effect P-Values,Const R,I Var R,Fixed Effect P-Values,Const FE,I Var FE,Time Effect P-Values,Const FT,I Var FT,Entity and Time P-Values,Const FA,I Var FA,Hausman Entity vs Random,Hausman Time vs Random,Hausman Entity and Time vs Random,Hausman Time vs Entity,Hausman Entity and Time vs Entity,Hausman Entity and Time vs Time
0,PSEGDP,Suicide_per_100000,3.657643e-10,4.381923e-11,1.529498,0.002125,17.917527,-0.156435,0.001401,18.120884,-0.166899,0.371461,13.939831,0.048227,0.328157,13.636638,0.063827,0.662132,8.673253e-28,5.456638e-07,3.776776e-50,3.458074e-08,0.913563
1,PSEGDP,Male,1.906161e-07,8.035168e-08,1.510196,0.002290,27.854841,-0.245792,0.001869,28.061072,-0.256403,0.991943,23.095622,-0.000918,0.267228,20.884388,0.112855,0.825321,6.830906e-08,6.463922e-08,5.001181e-10,5.708909e-09,0.043721
2,PSEGDP,Female,2.560213e-24,1.714770e-17,1.652448,0.012941,8.596508,-0.073210,0.005290,8.828994,-0.085172,0.002029,5.569696,0.082528,0.519295,6.676525,0.025578,0.321986,1.000000e+00,1.043453e-03,1.000000e+00,7.716332e-05,0.153211
3,Old_Age,Suicide_per_100000,3.978418e-01,1.747923e-01,1.536632,0.005544,16.775360,-0.277943,0.002854,16.968499,-0.306223,0.005845,12.828286,0.300000,0.021124,12.854631,0.296143,0.432537,3.031250e-41,7.496375e-12,1.653920e-61,5.895776e-14,0.998399
4,Old_Age,Male,1.069319e-01,5.770374e-01,1.510636,0.006936,26.002199,-0.428203,0.003884,26.271861,-0.467688,0.006067,19.642009,0.503076,0.009519,19.535319,0.518698,0.472924,3.492362e-23,6.509758e-14,1.380790e-28,4.032098e-16,0.981053
